# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
doc=ET.parse('./data/mondial_database.xml')

## 1) 10 Countries with lowest Infant mortality rates

In [6]:
All=[]
#This will look for country names and where infant_mortality is
for element in doc.iterfind('country'):
    country_list=element.find('name')

    infant_mortality=element.find('infant_mortality') 
    
#This will extract country name and infant mortality on available entries
    if country_list != None:
        if infant_mortality != None:    

# Country name and infant_mortality data were saved in a list named "All"
            All.append([country_list.text, infant_mortality.text])
            

#All list is converted to a dataframe to easily work with numeric columns   
df=pd.DataFrame(All, columns=['Country', 'Infant Mortality'])

#Infant Mortality column is not numeric and needed to be converted to numberic
df['Infant Mortality']=pd.to_numeric(df['Infant Mortality'])

#Country ranked to 10 lowest infant mortality
lowest10=df.set_index('Country').sort_values(by='Infant Mortality').head(10)
lowest10

,Infant Mortality
Country,
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13


## 2) 10 Cities with largest population

In [7]:
all_list=[]

#This will find country name and all city and provinces that has city names on it as well
for element in doc.iterfind('country'):
    country=element.find('name').text
    city_list=element.findall('city')
    prov_list=element.findall('province')

# This will collect all cities that are under provinces and save it to city_list too
    for prov in prov_list:
        city_list +=prov.findall('city')

#This will find the name of city, latest population and country code of that city
    for city in city_list:
        city_name=city.find('name').text
        pop=city.find('.//population[last()]')
        coun_code=city.attrib['country']
        
# This extract city population and year on available entries   
        if pop !=None:
            city_pop=int(pop.text)
            year=pop.get('year')

# Country name, city name, year, population will be saved in all_list            
        all_list.append([country, city_name, year, city_pop])

# all_list converted to citipop data frame
citipop=pd.DataFrame(all_list, columns=['Country', 'City', 'Year', 'Population'])

#DataFrame sorted from highest population to lowest and only shows top 10
citipop.sort_values(by='Population', ascending=False).head(10)



   

,Country,City,Year,Population
1341,China,Shanghai,2010,22315474
771,Turkey,Istanbul,2012,13710512
1527,India,Mumbai,2011,12442373
479,Russia,Moskva,2013,11979529
1340,China,Beijing,2010,11716620
2810,Brazil,São Paulo,2010,11152344
1342,China,Tianjin,2010,11090314
1064,China,Guangzhou,2010,11071424
1582,India,Delhi,2011,11034555
1067,China,Shenzhen,2010,10358381


## 3) 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [8]:
etall=[]
allpop=[]
# This will find country name
for elem in doc.iterfind('country'):
    con = elem.find('name').text
    pop=elem.find('population')

#This will find Ethnic group and % of Ethnic group in a given country
    for e in elem.iterfind('ethnicgroup'):
        grp= e.text
        per = float(e.attrib['percentage'])

# Country, ethnic group and its percentage saved in etall list        
        etall.append([con,grp,per]) 
    
# This will return year and population present in a given country
    for y in elem.iterfind('population'):
        year=int(y.attrib['year'])
        pop=int(y.text)

# country name, year and population saved in allpop list
        allpop.append([con, year, pop])


#etall list converted to dataFrame dfet
dfet=pd.DataFrame(etall, columns=['Country', 'Ethnic Group', 'Percentage'])

#allpop list converted to dataFrame dfpop    
dfpop=pd.DataFrame(allpop, columns=['Country', 'Year', 'Population'])  

#only the latest year on each country are extracted from dfpop
idx=dfpop.groupby(['Country'])['Year'].transform(max)==dfpop['Year']
dfpop=dfpop[idx]

#dfpop and dfet dataframe are merged on entries present on both. 
#Country columns present on both used to merge both DFs
Etpop=dfpop.merge(dfet, how='inner')

#Ethnicity percentage multiplied by population of the given country and saved in "Ethnic Group Population" column
Etpop['Ethnic Group Population']=Etpop.Percentage/100*Etpop.Population

#Sum of Each ethnic group is calculated and sorted from highest to lowest and returns only top 10 highest
Etpop.groupby(['Ethnic Group'])['Ethnic Group Population'].sum().sort_values(ascending=False).head(10)


Ethnic Group
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: Ethnic Group Population, dtype: float64

## 4)name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [9]:

country_code=[]
for elem in doc.iterfind('country'):
    country_name=elem.find('name').text
    code=elem.attrib['car_code']
    country_code.append([country_name, code])
dfcountry_code=pd.DataFrame(country_code, columns=['Country', 'Country_Code'])
dfcountry_code.set_index('Country')



,Country_Code
Country,
Albania,AL
Greece,GR
Macedonia,MK
Serbia,SRB
Montenegro,MNE
Kosovo,KOS
Andorra,AND
France,F
Spain,E


 ## Longest River

In [10]:
river_list=[]
#This will find the name of river, length and country code of river   
for river in doc.iterfind('river'):
    river_name=river.find('name').text
    river_length=river.find('length')
    con=river.attrib['country']

# This will extract river length on available entries
    if river_length !=None:
        river_length=river.find('length').text

#Country name, river name and length saved in river_list        
    river_list.append([con, river_name, river_length])
    
#river_list converted to dfriver dataFrame
dfriver=pd.DataFrame(river_list, columns=['Country_Code', 'River_Name', 'Length'])

#Length column converted to numeric
dfriver['Length']=pd.to_numeric(dfriver.Length)

#DataFrame sorted from highest length to lowest and returns top 1
Longest=dfriver.sort_values(by='Length', ascending=False).head(1)
Longest


,Country_Code,River_Name,Length
174,CO BR PE,Amazonas,6448.0


In [68]:
lo=Longest.loc[:, ['River_Name', 'Length']]
lo

,River_Name,Length
174,Amazonas,6448.0


In [79]:
#This will separate the Country Codes into different rows but will have different column names
riv=pd.concat([pd.Series(row['River_Name'], row['Country_Code'].split(' '))              
                    for _, row in Longest.iterrows()]).reset_index()

#This will rename columns
riv.columns=['Country_Code', 'River_Name']

# This will result in columns with country_code, river name, length and country
riv=riv.merge(lo).merge(dfcountry_code, how='inner')
riv




,Country_Code,River_Name,Length,Country
0,CO,Amazonas,6448.0,Colombia
1,BR,Amazonas,6448.0,Brazil
2,PE,Amazonas,6448.0,Peru


## Largest Lake

In [87]:
lake_list=[]

#This will find lake name area and country location 
for lake in doc.iterfind('lake'):
    lake_name=lake.find('name').text
    lake_area=lake.find('area')
    cont=lake.attrib['country']

# This will extract lake area on available entries
    if lake_area !=None:
        lake_area=lake.find('area').text
        
#country name, lake name and lake area saved on lake_list        
    lake_list.append([cont, lake_name, lake_area])

#lake_list converted to DataFrame dflake
dflake=pd.DataFrame(lake_list, columns=['Country_Code', 'Lake_Name', 'Area'])

# Area column converted to numeric 
dflake['Area']=pd.to_numeric(dflake.Area)

#DataFrame sorted from highest ot lowest and returns top 1
lar=dflake.sort_values(by='Area', ascending=False).head(1)
lar

,Country_Code,Lake_Name,Area
54,R AZ KAZ IR TM,Caspian Sea,386400.0


In [86]:

ke=lar.loc[:, ['Lake_Name', 'Area']]

#This will separate the Country Codes into different rows but will have different column names
la=pd.concat([pd.Series(row['Lake_Name'], row['Country_Code'].split(' '))              
                    for _, row in dflake.iterrows()]).reset_index()

#This will rename columns
la.columns=['Country_Code', 'Lake_Name']

# This will result in columns with country_code, lake name, area and country
la=la.merge(ke).merge(dfcountry_code, how='inner')
la




,Country_Code,Lake_Name,Area,Country
0,R,Caspian Sea,386400.0,Russia
1,AZ,Caspian Sea,386400.0,Azerbaijan
2,KAZ,Caspian Sea,386400.0,Kazakhstan
3,IR,Caspian Sea,386400.0,Iran
4,TM,Caspian Sea,386400.0,Turkmenistan


## Highest Elevation Airport

In [14]:
airport_list=[]
# This will find airport name, elevation and country code
for airport in doc.iterfind('airport'):
    airport_name=airport.find('name').text
    elevation=airport.find('elevation')
    cont=airport.attrib['country']
    
# Elevation is extracted on available entries
    if elevation !=None:
        elevation=airport.find('elevation').text

#Country name, airport name and elevation saved on airport_list
    airport_list.append([cont, airport_name, elevation])
    
#airport_list converted to dataframe dfairport
dfairport=pd.DataFrame(airport_list, columns=['Country_Code', 'Airport_Name', 'Elevation'])

#Elevation columns converted to numeric
dfairport['Elevation']=pd.to_numeric(dfairport.Elevation)

#DataFrame sorted from higest to lowest elevation and retuns only top 1 & merged with dfcountry_code dataframe
dfairport.sort_values(by='Elevation', ascending=False).head(1).merge(dfcountry_code, how='inner')


,Country_Code,Airport_Name,Elevation,Country
0,BOL,El Alto Intl,4063.0,Bolivia
